In [1]:
import json
import pandas as pd
from functools import reduce
import numpy as np

In [2]:
with open('train.json', 'r') as raw_data:
    data = json.load(raw_data)
    df = pd.DataFrame(data)

In [4]:
texts = [['i', 'have', 'a', 'cat'],['he', 'have', 'a', 'dog'],['he', 'and', 'i', 'have', 'a', 'cat', 'and', 'a', 'dog']]
dictionary = list(enumerate(set(reduce(lambda x, y: x+y, texts))))

In [5]:
def vectorize(text):
    vector = np.zeros(len(dictionary))
    for i, word in dictionary:
        num = 0
        for w in text:
            if w == word:
                num += 1
        if num:
            vector[i] = num
    return vector

for t in texts:
    print(vectorize(t))
        

[1. 0. 1. 0. 0. 1. 1.]
[0. 1. 1. 0. 1. 0. 1.]
[1. 1. 1. 2. 1. 1. 2.]


Используя алгоритмы вроде Вag of Words, мы теряем порядок слов в тексте, а значит, тексты "i have no cows" и "no, i have cows" будут идентичными после векторизации, хотя и противоположными семантически. Чтобы избежать этой проблемы, можно сделать шаг назад и изменить подход к токенизации: например, использовать N-граммы (комбинации из N последовательных терминов).

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
vect = CountVectorizer(ngram_range=(1,1)) 

In [ ]:
vect.fit_transform(['no i have cows', 'i have no cows']).toarray()

In [ ]:
vect.vocabulary_

In [ ]:
vect = CountVectorizer(ngram_range=(1,2))
vect.fit_transform(['no i have cows', 'i have no cows']).toarray()

In [ ]:
vect.vocabulary_

In [ ]:
from scipy.spatial.distance import euclidean
vect = CountVectorizer(ngram_range=(3,3), analyzer='char_wb') 

In [ ]:
n1, n2, n3, n4 = vect.fit_transform(['иванов', 'петров', 'петренко', 'смит']).toarray()

In [ ]:
euclidean(n1, n2)

In [ ]:
euclidean(n2, n3)

In [ ]:
euclidean(n3, n4)

Слова, которые редко встречаются в корпусе (во всех рассматриваемых документах этого набора данных), но присутствуют в этом конкретном документе, могут оказаться более важными. Тогда имеет смысл повысить вес более узкотематическим словам, чтобы отделить их от общетематических. Этот подход называется TF-IDF

Word2Vec является частным случаем алгоритмов Word Embedding. Используя Word2Vec и подобные модели, мы можем не только векторизовать слова в пространство большой размерности (обычно несколько сотен), но и сравнивать их семантическую близость. Классический пример операций над векторизированными представлениями: king – man + woman = queen.

In [ ]:
from keras.applications.resnet50 import ResNet50
from keras.preprocessing import image
from scipy.misc import face
import numpy as np
import pytesseract

In [ ]:
import reverse_geocoder as revgc
from PIL import Image
import requests
from io import BytesIO

In [ ]:
df['dow'] = df['created'].apply(lambda x: x.date().weekday())
df['is_weekend'] = df['created'].apply(lambda x: 1 if x.date().weekday() in (5, 6) else 0)

In [ ]:
from sklearn.preprocessing import StandardScaler  
from scipy.stats import beta
from scipy.stats import shapiro
data = beta(1, 10).rvs(1000).reshape(-1, 1)

In [ ]:
shapiro(data)

In [ ]:
 shapiro(StandardScaler().fit_transform(data))

In [ ]:
data = np.array([1, 1, 0, -1, 2, 1, 2, 3, -2, 4, 100]).reshape(-1, 1).astype(np.float64)

In [ ]:
StandardScaler().fit_transform(data)

In [ ]:
from sklearn.preprocessing import MinMaxScaler
MinMaxScaler().fit_transform(data)

In [ ]:
(data - data.min()) / (data.max() - data.min())

In [ ]:
from scipy.stats import lognorm
data = lognorm(s=1).rvs(1000)
shapiro(data)

In [ ]:
 shapiro(np.log(data))

In [ ]:
import statsmodels.api as sm

# возьмем признак price из датасета Renthop и пофильтруем руками совсем экстремальные значения для наглядности
price = df.price[(df.price <= 20000) & (df.price > 500)]
price_log = np.log(price)
price_mm = MinMaxScaler().fit_transform(price.values.reshape(-1, 1).astype(np.float64)).flatten()
# много телодвижений, чтобы sklearn не сыпал warning-ами
price_z = StandardScaler().fit_transform(price.values.reshape(-1, 1).astype(np.float64)).flatten()
sm.qqplot(price_log, loc=price_log.mean(), scale=price_log.std()).savefig('qq_price_log.png')
sm.qqplot(price_mm, loc=price_mm.mean(), scale=price_mm.std()).savefig('qq_price_mm.png')

In : sm.qqplot(price_z, loc=price_z.mean(), scale=price_z.std()).savefig('qq_price_z.png')

In [7]:
rooms = df["bedrooms"].apply(lambda x: max(x, .5))

In [8]:
df["price_per_bedroom"] = df["price"] / rooms

Заполнение пропусков: pandas.DataFrame.fillna и sklearn.preprocessing.Imputer.

In [9]:
from sklearn.feature_selection import VarianceThreshold
from sklearn.datasets import make_classification
x_data_generated, y_data_generated = make_classification()
x_data_generated.shape

(100, 20)

In [10]:
VarianceThreshold(.7).fit_transform(x_data_generated).shape

(100, 18)

In [11]:
VarianceThreshold(.8).fit_transform(x_data_generated).shape

(100, 16)

In [12]:
VarianceThreshold(.9).fit_transform(x_data_generated).shape

(100, 13)

In [13]:
from sklearn.datasets import make_classification
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectFromModel
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import make_pipeline

x_data_generated, y_data_generated = make_classification()

pipe = make_pipeline(SelectFromModel(estimator=RandomForestClassifier()),
                     LogisticRegression())

lr = LogisticRegression()
rf = RandomForestClassifier()

print(cross_val_score(lr, x_data_generated, y_data_generated, scoring='neg_log_loss').mean())
print(cross_val_score(rf, x_data_generated, y_data_generated, scoring='neg_log_loss').mean())
print(cross_val_score(pipe, x_data_generated, y_data_generated, scoring='neg_log_loss').mean())

-0.27653575412182435
-0.20712549726741009
-0.25761434385839427


In [14]:
x_data, y_data = get_data()
x_data = x_data.values

pipe1 = make_pipeline(StandardScaler(),
                      SelectFromModel(estimator=RandomForestClassifier()),
                      LogisticRegression())

pipe2 = make_pipeline(StandardScaler(),
                      LogisticRegression())

rf = RandomForestClassifier()

print('LR + selection: ', cross_val_score(pipe1, x_data, y_data, scoring='neg_log_loss').mean())
print('LR: ', cross_val_score(pipe2, x_data, y_data, scoring='neg_log_loss').mean())
print('RF: ', cross_val_score(rf, x_data, y_data, scoring='neg_log_loss').mean())


NameError: name 'get_data' is not defined